<a href="https://colab.research.google.com/github/alexk2206/tds_capstone/blob/main/QA_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Generation of the Q and A Dataset**

In [13]:
import pandas as pd

url ="https://raw.githubusercontent.com/alexk2206/tds_capstone/refs/heads/main/answer_combinations.json"

q_changed_options = pd.read_json(url)
q_changed_options

,question,type,options
0,Data processing consent,SINGLE_SELECT,[Yes]
1,Data processing consent,SINGLE_SELECT,[No]
2,Customer group,SINGLE_SELECT,[End User]
3,Customer group,SINGLE_SELECT,[Wholesaler]
4,Customer group,SINGLE_SELECT,[Distributor]
...,...,...,...
8530,Searches a solution for,MULTI_SELECT,"[Clean up CRM, Extract data from emails, Impro..."
8531,Searches a solution for,MULTI_SELECT,"[Scan business cards, Clean up CRM, Extract da..."
8532,Next steps,SINGLE_SELECT,[Offer]
8533,Next steps,SINGLE_SELECT,[Meeting]


In [28]:
import google.generativeai as genai
from IPython.display import display, Markdown
from google.colab import userdata
import time

# retrieving the key stored in Colab
key = userdata.get('GOOGLE_API_KEY')

# configure the key for calling GenAI model
genai.configure(api_key=key)

# load model
model = genai.GenerativeModel("gemini-1.5-flash")

In [68]:
def sc_generate_answer(question, options):
  prompt = f"""
  You are being asked a question and must answer in a way that sounds like you're speaking in the first person.
  Your response should be varied and sound natural, like you're being interviewed.
  Answer in present tense like you are having a conversation.
  Question: {question}\n
  Answer: {options}\n
  Answer as a sentence:
  """
  response = model.generate_content(
      contents = prompt,
      generation_config = genai.GenerationConfig(
          max_output_tokens=100,
          temperature=2,)
    )

  answer = response.text.strip()

  time.sleep(3)

  return answer

In [70]:
q_changed_options['context'] = q_changed_options.head(5).apply(lambda row: sc_generate_answer(row['question'], row['options']), axis=1)

In [71]:
q_changed_options

,question,type,options,context
0,Data processing consent,SINGLE_SELECT,[Yes],"Oh yeah, absolutely. I give my consent for da..."
1,Data processing consent,SINGLE_SELECT,[No],"No, I don't consent to my data being processed."
2,Customer group,SINGLE_SELECT,[End User],I'm mainly focused on the end user.
3,Customer group,SINGLE_SELECT,[Wholesaler],"I mainly work with wholesalers, that's my prim..."
4,Customer group,SINGLE_SELECT,[Distributor],"Well, right now, my main customer group is dis..."
...,...,...,...,...
8530,Searches a solution for,MULTI_SELECT,"[Clean up CRM, Extract data from emails, Impro...",NaN
8531,Searches a solution for,MULTI_SELECT,"[Scan business cards, Clean up CRM, Extract da...",NaN
8532,Next steps,SINGLE_SELECT,[Offer],NaN
8533,Next steps,SINGLE_SELECT,[Meeting],NaN
